In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
dataset_path = '/content/drive/MyDrive/New'

files = os.listdir(dataset_path)
print("Files in dataset directory:",files)

Files in dataset directory: ['SampleSubmission.csv', 'License Plate Recognition.docx', 'Licplatesdetection_train.csv', 'Licplatesrecognition_train.csv', 'Licplatesrecognition_train.zip', 'Licplatesdetection_train.zip', 'test.zip']


In [ ]:
csv_count = 0
zip_count = 0

for file in files:
    if file.endswith('.csv'):
        csv_count += 1
    elif file.endswith('.zip'):
        zip_count += 1

print(f"Number of CSV files: {csv_count}")
print(f"Number of ZIP files: {zip_count}")


Number of CSV files: 3
Number of ZIP files: 3


In [ ]:
import zipfile

def extract_zip_(file_path, extract_to):
    with zipfile.ZipFile(file_path, 'r') as zip_ref:
         zip_ref.extractall(extract_to)
         print(f"Extracted: {file_path} to {extract_to}")

for file in files:
  if file.endswith('.zip'):
    file_path = os.path.join(dataset_path, file)
    extract_to = os.path.join(dataset_path, file.replace('.zip', ''))
    extract_zip_(file_path, extract_to)


Extracted: /content/drive/MyDrive/New/Licplatesrecognition_train.zip to /content/drive/MyDrive/New/Licplatesrecognition_train
Extracted: /content/drive/MyDrive/New/Licplatesdetection_train.zip to /content/drive/MyDrive/New/Licplatesdetection_train
Extracted: /content/drive/MyDrive/New/test.zip to /content/drive/MyDrive/New/test


#Installing Required Libraries

In [ ]:
!apt-get install -y tesseract-ocr
!pip install pytesseract
!pip install opencv-python-headless tensorflow keras numpy pandas matplotlib imutils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 29 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (4,967 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 125044 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [ ]:
!pip install --upgrade tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 844.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 17.4 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.18.0
    Uninstalling tensorboard-2.18.0:
      Successfully uninstalled tensorboard-2.18.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.18.0
    Uninstalling tensorflow-2.18.0:
      Successfully uninstalled tensorflow-2.18.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.18.0 requires tensorflow<2.19,>=2.18, but you have tensorflow 2.19.0

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.18.0


In [ ]:
import os
import cv2
import pytesseract
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from imutils import paths
from sklearn.preprocessing import LabelEncoder

#Data Preparation

In [ ]:
# Load and Preprocess Images
image_dir = '/content/drive/MyDrive/your_image_directory/'
# Assign the result of list_images to image_paths
image_paths = list(paths.list_images(image_dir)) # this line is added to fix the error
def preprocess_image(image_path):
  image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
  image = cv2.resize(image, (128, 64))
  image = image / 255.0
  image = np.expand_dims(image, axis=-1)
  return image

image = np.array([preprocess_image(img)
for img in image_paths])

In [ ]:
labels = pd.read_csv("/content/drive/MyDrive/New/Licplatesdetection_train.csv")
print(labels.head())
labels = pd.read_csv("/content/drive/MyDrive/New/Licplatesrecognition_train.csv")
print(labels.head())

    img_id  ymin  xmin  ymax  xmax
0    1.jpg   276    94   326   169
1   10.jpg   311   395   344   444
2  100.jpg   406   263   450   434
3  101.jpg   283   363   315   494
4  102.jpg   139    42   280   222
    img_id      text
0    0.jpg  117T3989
1    1.jpg  128T8086
2   10.jpg   94T3458
3  100.jpg  133T6719
4  101.jpg   68T5979


In [ ]:
detection_labels = pd.read_csv("/content/drive/MyDrive/New/Licplatesdetection_train.csv")
recognition_labels = pd.read_csv("/content/drive/MyDrive/New/Licplatesrecognition_train.csv")

In [ ]:
import string

characters = string.ascii_uppercase + string.digits
char_to_int = {char: i for i, char in enumerate(characters, start=1)}

def encode_plate_text(text):
  return [char_to_int[char] for char in text if char in char_to_int]

recognition_labels["encoded_text"] = recognition_labels["text"].apply(encode_plate_text)
print(recognition_labels.head())

    img_id      text                      encoded_text
0    0.jpg  117T3989  [28, 28, 34, 20, 30, 36, 35, 36]
1    1.jpg  128T8086  [28, 29, 35, 20, 35, 27, 35, 33]
2   10.jpg   94T3458      [36, 31, 20, 30, 31, 32, 35]
3  100.jpg  133T6719  [28, 30, 30, 20, 33, 34, 28, 36]
4  101.jpg   68T5979      [33, 35, 20, 32, 36, 34, 36]


## Data Preprocessing

In [ ]:
# Image Preprocessing
from tensorflow.keras.preprocessing.image import img_to_array

IMG_SIZE = (224, 224)
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, IMG_SIZE)
    img = img.astype("float32") / 255.0
    img = img_to_array(img)
    return img


# Train a License Plate Detection model

In [ ]:
import tensorflow as tf

model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
x = tf.keras.layers.GlobalAveragePooling2D()(model.output)
x = tf.keras.layers.Dense(256, activation="relu")(x)
x = tf.keras.layers.Dense(4, activation="linear")(x)
detection_model = tf.keras.Model(inputs=model.input, outputs=x)
detection_model.compile(optimizer="adam", loss="mse")

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


# Train a License Plate OCR Model

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Reshape, Flatten, Dense, LSTM, Bidirectional

input_layer = Input(shape=(32, 128, 1))
x = Conv2D(64, (3, 3), activation="relu", padding="same")(input_layer)
x = MaxPooling2D(pool_size=(2,2))(x)

timesteps = x.shape[1]
features = x.shape[2] * x.shape[3]
x = Reshape((timesteps, features))(x)

x = Bidirectional(LSTM(128, return_sequences=True))(x)
x = Dense(len(char_to_int) + 1, activation="softmax")(x)

ocr_model = Model(inputs=input_layer, outputs=x)
ocr_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Model Evaluation

In [ ]:
from tensorflow.keras.metrics import Accuracy

y_true = [0, 1, 1, 0]
y_pred = [0, 1, 0, 0]
def iou_score(true_box, pred_box):
    pass

accuracy_metric = Accuracy()
accuracy_metric.update_state(y_true, y_pred)
print("Recognition Accuracy:", accuracy_metric.result().numpy())


Recognition Accuracy: 0.75


# Deployment

In [34]:
from flask import Flask, request, jsonify
import cv2

app = Flask(__name__)
@app.route('/predict', methods=['POST'])
def predict():
    file = request.files['image']
    image = preprocess_image(file)
    bbox = detection_model.predict(image)
    plate_text = ocr_model.predict(image)
    return jsonify({"bbox":bbox.tolist(), "text":plate_text.tolist()})

if __name__== '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
